## <span style="color:blue">Introduction to computation in physical sciences</span>
### J Wang and A Wang, [github.com/com-py/intro](https://github.com/com-py/intro) 
### Ch09, `ch09`, Game of life, chaos

In [1]:
import numpy as np

def update_grid(grid):
    n_row, n_col = grid.shape
    next_grid = np.zeros(grid.shape, dtype=int)
    for i in range(1, n_row-1):
        for j in range(1, n_col-1):
            cell = grid[i, j]
            neighborhood = grid[i-1:i+2, j-1:j+2]
            n_alive_neighbors = np.sum(neighborhood) - cell
            next_grid[i, j] = update_cell(cell, n_alive_neighbors)
    return next_grid

In [2]:
def update_cell(cell, n_alive_neighbors):
    if cell:
        return(int(2 <= n_alive_neighbors <= 3))
    else:
        return(int(n_alive_neighbors == 3))

In [3]:
grid = np.random.randint(low=0, high=2, size=(20, 20))
grid = np.pad(grid, pad_width=1, mode='constant', constant_values=0)

In [4]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import colors

def play(grid, T=100, show=True):
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.set(xticks=[], yticks=[], aspect='equal')
    ims = []
    for _ in range(T + 1):
        im = ax.pcolormesh(grid, edgecolor='k', linewidth=.01, animated=True,
                           cmap=colors.ListedColormap(['white', 'green']))
        ims.append([im])
        grid = update_grid(grid)
    anim = animation.ArtistAnimation(fig, ims, interval=100, blit=True)
    plt.show() if show else plt.close()
    return anim

In [5]:
np.random.seed(1)
grid = np.random.randint(low=0, high=2, size=(20, 20))
grid = np.pad(grid, pad_width=1, mode='constant', constant_values=0)
anim = play(grid, T=110, show=False)

from IPython.display import HTML
HTML(anim.to_jshtml())

In [6]:
class Road:
    def __init__(self, length, density, p_slow=.1, v_max=5):
        n_car = int(np.round(length * density))
        xs = sorted(np.random.choice(length, size=n_car, replace=False))
        vs = np.zeros(n_car, dtype=int)
        self.cars = [(xs[i], vs[i]) for i in range(n_car)]
        self.n_car, self.length, self.density = n_car, length, density
        self.p_slow, self.v_max = p_slow, v_max

    def update(self):
        updated_cars = []
        for i in range(self.n_car):
            x, v = self.cars[i]
            front_x, front_v = self.cars[(i+1) % self.n_car]
            dist = (front_x - 1 - x) % self.length
            if v < self.v_max: v += 1
            if dist < v: v = dist
            if v > 0 and np.random.rand() < self.p_slow: v -= 1
            updated_cars.append(((x + v) % self.length, v))
        self.cars = updated_cars

In [7]:
road = Road(length=100, density=.05)
road.update()

In [8]:
def logistic_map(x, r):
    return 4*r * x * (1-x)

In [9]:
import sympy as sp
x, r = sp.symbols('x r')
sp.solveset(logistic_map(x, r) - x, x)

FiniteSet(0, (4*r - 1)/(4*r))

In [10]:
import ipywidgets

def cobweb_coords(x0, r, T=20):
    x1 = logistic_map(x0, r)
    coords = [(x0, 0), (x0, x1)]
    for _ in range(T):
        x0, x1 = x1, logistic_map(x1, r)
        coords.extend([(x0, x0), (x0, x1)])
    return coords

@ipywidgets.interact(x0=ipywidgets.FloatSlider(min=0, max=1, step=.01),
                     r=ipywidgets.FloatSlider(min=0, max=1, step=.01))
def evolution_plots(x0, r):
    web_xs, web_ys = list(zip(*cobweb_coords(x0, r)))
    traj = [x for i, x in enumerate(web_xs) if i % 2 == 0]
    xs = np.arange(0, 1, .01)

    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(7, 3), sharey=True)
    axs[0].plot(traj, linestyle='-', marker='.')
    axs[0].set(xlabel='$t$', ylabel='$x_t$', ylim=[0, 1])
    axs[1].plot([0, 1], [0, 1], color='C0')
    axs[1].plot(xs, logistic_map(xs, r), color='C0')
    axs[1].plot(web_xs, web_ys, color='C1')
    axs[1].plot(web_xs[-1], web_ys[-1], color='black', marker='o')
    axs[1].set(xlabel='$x_t$', ylabel='$x_{t+1}$', xlim=[0, 1])

interactive(children=(FloatSlider(value=0.0, description='x0', max=1.0, step=0.01), FloatSlider(value=0.0, des…

In [11]:
sp.init_printing(use_latex='mathjax')  # For pretty printing
solset = sp.solveset(logistic_map(x, r) - x, x)
for sol in solset:
    display({sol: logistic_map(x, r).diff(x).subs(x, sol).simplify()})

{0: 4⋅r}

⎧4⋅r - 1         ⎫
⎨───────: 2 - 4⋅r⎬
⎩  4⋅r           ⎭

In [12]:
def bifurcation_plot(rs=np.arange(0, 1, .001), n_toss=1000, n_keep=100):
    fig, ax = plt.subplots()
    ax.set(xlabel='$r$', ylabel='$x$')
    for r in rs:
        x, xs_keep = 0.5, []
        for _ in range(n_toss): x = logistic_map(x, r)
        for _ in range(n_keep):
            x = logistic_map(x, r)
            xs_keep.append(x)
        ax.plot([r]*n_keep, xs_keep, 'C0,')

In [13]:
class SimpleNetwork:
    def __init__(self, w1, w2, b):
        self.w1, self.w2, self.b = w1, w2, b

    def predict(self, x1, x2):
        return int(self.w1*x1 + self.w2*x2 + self.b > 0)

net = SimpleNetwork(w1=1, w2=2, b=-2.5)
net.predict(x1=1, x2=1)

1

In [14]:
from tensorflow import keras

def add_grid_padding(grid):
    return np.pad(grid, pad_width=1, mode='constant', constant_values=0)

def remove_grid_padding(grid):
    return grid[1:-1, 1:-1]
  
def generate_data(n_sample, n_row, n_col):
    X, Y = [], []
    for _ in range(n_sample):
        grid = add_grid_padding(
            np.random.randint(low=0, high=2, size=(n_row, n_col)))
        updated_grid = update_grid(grid)
        X.append(remove_grid_padding(grid))
        Y.append(remove_grid_padding(updated_grid))
    return np.expand_dims(X, -1), np.expand_dims(Y, -1)

keras.utils.set_random_seed(1)
n_row, n_col = 20, 20
X_train, Y_train = generate_data(n_sample=1000, n_row=n_row, n_col=n_col)
X_train.shape

(1000, 20, 20, 1)

In [15]:
input_shape = (n_row, n_col, 1)  
model = keras.Sequential([
    keras.Input(shape=input_shape),
    keras.layers.Dense(units=100, activation='relu'),
    keras.layers.Dense(units=100, activation='relu'),
    keras.layers.Dense(units=1, activation='sigmoid')
])

In [16]:
SGD = keras.optimizers.SGD(learning_rate=1)
model.compile(loss='mse', optimizer=SGD, metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=10, verbose=1)

Epoch 1/10
100/100 [==============================] - 1s 8ms/step - loss: 0.2104 - accuracy: 0.6888
Epoch 2/10
100/100 [==============================] - 1s 8ms/step - loss: 0.2084 - accuracy: 0.6896
Epoch 3/10
100/100 [==============================] - 1s 9ms/step - loss: 0.2084 - accuracy: 0.6896
Epoch 4/10
100/100 [==============================] - 1s 9ms/step - loss: 0.2084 - accuracy: 0.6896
Epoch 5/10
100/100 [==============================] - 1s 9ms/step - loss: 0.2084 - accuracy: 0.6896
Epoch 6/10
100/100 [==============================] - 1s 9ms/step - loss: 0.2084 - accuracy: 0.6896
Epoch 7/10
100/100 [==============================] - 1s 8ms/step - loss: 0.2084 - accuracy: 0.6896
Epoch 8/10
100/100 [==============================] - 1s 8ms/step - loss: 0.2084 - accuracy: 0.6896
Epoch 9/10
100/100 [==============================] - 1s 8ms/step - loss: 0.2084 - accuracy: 0.6896
Epoch 10/10
100/100 [==============================] - 1s 8ms/step - loss: 0.2084 - accuracy: 0.6896

In [17]:
np.unique(model.predict(X_train))

array([0.22743931, 0.36943513], dtype=float32)

In [18]:
np.mean(Y_train == 0)

0.6896475

In [19]:
model = keras.Sequential([
    keras.Input(shape=input_shape),
    keras.layers.Conv2D(filters=10, kernel_size=(5, 5), activation='relu', 
                        padding='same'),
    keras.layers.Dense(units=1, activation='sigmoid')
])
model.compile(loss='mse', optimizer=SGD, metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=10, verbose=1)

Epoch 1/10
100/100 [==============================] - 1s 3ms/step - loss: 0.1688 - accuracy: 0.7185
Epoch 2/10
100/100 [==============================] - 0s 3ms/step - loss: 0.1421 - accuracy: 0.7248
Epoch 3/10
100/100 [==============================] - 0s 3ms/step - loss: 0.1174 - accuracy: 0.8025
Epoch 4/10
100/100 [==============================] - 0s 3ms/step - loss: 0.0918 - accuracy: 0.8758
Epoch 5/10
100/100 [==============================] - 0s 3ms/step - loss: 0.0548 - accuracy: 0.9566
Epoch 6/10
100/100 [==============================] - 0s 3ms/step - loss: 0.0295 - accuracy: 0.9841
Epoch 7/10
100/100 [==============================] - 0s 3ms/step - loss: 0.0098 - accuracy: 1.0000
Epoch 8/10
100/100 [==============================] - 0s 3ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 9/10
100/100 [==============================] - 0s 3ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 10/10
100/100 [==============================] - 0s 3ms/step - loss: 0.0028 - accuracy: 1.0000

In [20]:
X_test, Y_test = generate_data(n_sample=1000, n_row=n_row, n_col=n_col)
print(model.metrics_names)
print(model.evaluate(X_test, Y_test, verbose=0))

['loss', 'accuracy']
[0.002449536696076393, 1.0]
